# Sales Account Meeting Briefing

**User Story**: US1 — Meeting Preparation Assistant
**Persona**: Account Executive

This notebook demonstrates how to use the Salesforce AI Assistant to prepare
for account meetings. The assistant pulls live CRM data (account details, contacts,
open opportunities, recent activities) and generates a comprehensive briefing
with recommended talking points.

## Prerequisites
- `.env` file configured with Salesforce and Azure AI credentials
- `salesforce-crm` MCP server available
- Python packages installed (`pip install -r requirements.txt`)

In [ ]:
# Cell 2: Environment + Auth Setup
import os
import sys

# Add project root to path
sys.path.insert(0, os.path.abspath(".."))

from dotenv import load_dotenv

load_dotenv("../.env")

# Verify required env vars
required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing environment variables: {', '.join(missing)}")

print("✅ Environment configured")
print(f"   Project: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:50]}...")
print(f"   SF Instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: Start MCP Server + Connect with MCP Client
import asyncio
import subprocess
import time

import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()  # Allow nested event loops in Jupyter

# Start the salesforce-crm MCP server locally with SSE transport
MCP_PORT = 8100
mcp_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_crm.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(MCP_PORT)},
    cwd=os.path.abspath(".."),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
time.sleep(3)

if mcp_process.poll() is not None:
    stderr = mcp_process.stderr.read().decode() if mcp_process.stderr else ""
    raise RuntimeError(f"MCP server failed to start: {stderr}")

MCP_URL = f"http://127.0.0.1:{MCP_PORT}/sse"
print(f"✅ MCP server started (PID: {mcp_process.pid}) at {MCP_URL}")


# Connect to the MCP server via SSE and keep the session alive
async def _connect():
    """Open an SSE connection and MCP session, return (sse_cm, session_cm, session, tools)."""
    _sse_cm = sse_client(url=MCP_URL)
    read_stream, write_stream = await _sse_cm.__aenter__()
    _sess_cm = ClientSession(read_stream, write_stream)
    session = await _sess_cm.__aenter__()
    await session.initialize()
    tools_result = await session.list_tools()
    return _sse_cm, _sess_cm, session, tools_result.tools

_sse_cm, _sess_cm, mcp_session, mcp_tools = asyncio.get_event_loop().run_until_complete(_connect())

print(f"✅ MCP session connected — {len(mcp_tools)} tools available:")
for t in mcp_tools:
    print(f"   • {t.name}: {t.description[:80] if t.description else ''}")

In [ ]:
# Cell 4: Create OpenAI Client + Load System Prompt + Define Helper
import asyncio
import json
from pathlib import Path as _Path

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI

# Build an AzureOpenAI client using the base services endpoint.
# The project endpoint looks like https://<host>/api/projects/<name>;
# AzureOpenAI needs just https://<host>.
_project_endpoint = os.environ["AZURE_AI_PROJECT_ENDPOINT"]
_azure_endpoint = _project_endpoint.split("/api/")[0] if "/api/" in _project_endpoint else _project_endpoint

client = AzureOpenAI(
    azure_endpoint=_azure_endpoint,
    azure_ad_token_provider=get_bearer_token_provider(
        DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
    ),
    api_version="2024-10-21",
)

# Load the Sales agent system prompt
system_prompt = _Path("../agents/sales/system_prompt.md").read_text(encoding="utf-8")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")

# ---------- Convert MCP tools → OpenAI function-calling format ----------
openai_tools = []
for t in mcp_tools:
    openai_tools.append(
        {
            "type": "function",
            "function": {
                "name": t.name,
                "description": t.description or "",
                "parameters": t.inputSchema or {"type": "object", "properties": {}},
            },
        }
    )

# ---------- Helper: run a query with automatic MCP tool-calling loop ----------
async def _call_tool(name: str, arguments: dict) -> str:
    """Forward a tool call to the MCP server and return the text result."""
    result = await mcp_session.call_tool(name, arguments)
    parts = []
    for block in result.content:
        if hasattr(block, "text"):
            parts.append(block.text)
    return "\n".join(parts) if parts else ""


def chat(user_message: str, *, history: list | None = None, max_rounds: int = 10) -> list:
    """Send *user_message* to the model, automatically executing any MCP tool
    calls in a loop until the model produces a final text answer.

    Returns the updated message history (for multi-turn continuation).
    """
    if history is None:
        history = [{"role": "system", "content": system_prompt}]

    history.append({"role": "user", "content": user_message})

    loop = asyncio.get_event_loop()
    for _ in range(max_rounds):
        resp = client.chat.completions.create(
            model=MODEL,
            messages=history,
            tools=openai_tools,
            tool_choice="auto",
        )
        msg = resp.choices[0].message

        # Append the assistant message (may contain tool_calls)
        history.append(msg)

        if not msg.tool_calls:
            break  # Final text answer

        # Execute each tool call via MCP
        for tc in msg.tool_calls:
            args = json.loads(tc.function.arguments) if tc.function.arguments else {}
            tool_result = loop.run_until_complete(_call_tool(tc.function.name, args))
            history.append(
                {
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": tool_result,
                }
            )

    return history

print(f"✅ OpenAI client ready (model: {MODEL})")
print(f"✅ System prompt loaded ({len(system_prompt)} chars)")
print(f"✅ {len(openai_tools)} MCP tools mapped to OpenAI functions")

In [ ]:
# Cell 5: Discover accounts in your Salesforce org
import asyncio
import json as _json

_result = asyncio.get_event_loop().run_until_complete(
    mcp_session.call_tool("search_accounts", {"query": "%", "limit": 10})
)
_raw = "\n".join(b.text for b in _result.content if hasattr(b, "text"))
_data = _json.loads(_raw)

print("Accounts in your Salesforce org:")
print("-" * 50)
for _acct in _data.get("accounts", []):
    print(f"  • {_acct['name']}  (ID: {_acct['id']}, Industry: {_acct.get('industry', 'N/A')})")
print(f"\nTotal returned: {_data.get('total_count', 0)} | Has more: {_data.get('has_more', False)}")
print("\n👆 Copy one of these names into ACCOUNT_NAME in the next cell.")

In [ ]:
# Cell 5: Meeting Briefing Request
ACCOUNT_NAME = "GenePoint"  # Change to your target account

history = chat(
    f"Prepare me for my meeting with {ACCOUNT_NAME}. "
    f"Include account overview, key contacts, open opportunities, "
    f"recent activities, and suggested talking points."
)
print("✅ Briefing generated")

In [ ]:
# Cell 6: Display Briefing
from IPython.display import Markdown, display


def show(hist: list):
    """Display the last assistant message from history."""
    for m in reversed(hist):
        content = m["content"] if isinstance(m, dict) else m.content
        role = m.get("role") if isinstance(m, dict) else getattr(m, "role", None)
        if role == "assistant" and content:
            display(Markdown(content))
            return


show(history)

In [ ]:
# Cell 7: Follow-up Question (continues conversation via history)
history = chat(
    "Any other open opportunities with them? What's the total pipeline value?",
    history=history,
)
show(history)

In [ ]:
# Cell 8: Cleanup — close MCP session + stop server
import asyncio


async def _cleanup():
    try:
        await _sess_cm.__aexit__(None, None, None)
    except (RuntimeError, Exception):
        pass  # cancel-scope task mismatch is expected in Jupyter
    try:
        await _sse_cm.__aexit__(None, None, None)
    except (RuntimeError, Exception):
        pass

try:
    asyncio.get_event_loop().run_until_complete(_cleanup())
except (RuntimeError, Exception):
    pass

mcp_process.terminate()
mcp_process.wait(timeout=5)
print(f"✅ MCP session closed & server stopped (PID: {mcp_process.pid})")
print("\n--- Session Complete ---")